# Graphs

<img src="https://imgur.com/38QtUKm.png" style="width: 600px">


## Key Skills

- [P0] _Graph Representations_
  - Adjacency List/Map
  - Adjacency Matrix
- [P0] _Graph Traversal_
  - DFS
  - BFS
- [P0] _Graph Attributes_
  - Connected Components
  - Bipartite
  - Cycles
  - is/isn't Tree
  - is/isn't DAG
- Searching Graphs (Finding a path between two nodes)
  - Optimal Search
    - [P0] Dijkstra's Algorithm
    - [P1] A\* Search
- [P0] Topological Sort (Dependency Ordering)
  - Kahn's Algorithm
  - DFS
- [P1] Minimum Spanning Tree (Knowing the minimum cost to connect ALL nodes)
  - Prim's Algorithm
  - Kruskal's Algorithm
- [P1] Shortest Path (Knowing the minimum cost to connect TWO nodes)
  - Bellman-Ford Algorithm
  - Floyd-Warshall Algorithm
- [P2] Strongly Connected Components (Knowing which nodes are connected to each other)
  - Kosaraju's Algorithm
  - Tarjan's Algorithm
- [P2] Network Flow (Knowing the maximum flow between two nodes)
  - Ford-Fulkerson Algorithm
  - Edmonds-Karp Algorithm
  - Dinic's Algorithm
- [P1] Eulerian Path (Knowing the path that visits every edge exactly once)
  - Fleury's Algorithm
  - Hierholzer's Algorithm
- [P2] Hamiltonian Path (Knowing the path that visits every node exactly once)
  - Held-Karp Algorithm
  - Branch and Bound

## Graph Representations: Adjacency List/Map

We can easily build a graph from a list of edges; [u, v] where u is the source node and v is the destination node. We can use a dictionary to store the edges, where the key is the source node and the value is a list of destination nodes.


In [ ]:
from collections import defaultdict


def build_graph(edges):
    adj_map = defaultdict(list)  # {node: [adjacent nodes]}
    for u, v in edges:
        adj_map[u].append(v)
        adj_map[v].append(u)
    return adj_map

In some cases, we'll want to build an adjacency matrix instead of an adjacency list/map. The reasons for this would be that we're perhaps working with a dense graph, or we want to be able to quickly check if an edge exists between two nodes. We can use a 2D array to store the edges, where the row is the source node and the column is the destination node. This data structure is also useful for matrix multiplication, which we'll see later, and finally geographical type problems where we want some type of topology between nodes; nodes being some object (like land, or a city) and edges being some type of relationship between them (like distance, or a road, water, etc).

Let's see how we can build an adjacency matrix from a list of edges. We'll use a 2-D list.
Let's see the psudeo code for this algorithm first

```shell
FUNC build_adjacency_matrix(edges):
    GET all unique nodes from edges
    SORT all the unique nodes by NODE value
    CREATE a 2-D list of size len(unique_nodes) x len(unique_nodes)
    CREATE a NODE to INDEX map
    FOR each edge:
        GET source_index, destination_index
        SET the 2-D list at [source_node_index][destination_node_index] to 1
        SET the 2-D list at [destination_node_index][source_node_index] to 1
    RETURN the 2-D list
```

The major heavy lifting has to do with

1. getting all the unique nodes from the edges
2. sorting the unique nodes so that the columns and rows are in the same order

and below is the actual code implementation


In [ ]:
def build_adjacency_matrix(edges):
    nodes = set()
    for u, v in edges:
        nodes.add(u)
        nodes.add(v)
    nodes = sorted(list(nodes))
    n = len(nodes)
    adj_matrix = [[0 for _ in range(n)] for _ in range(n)]
    node_map = {node: i for i, node in enumerate(nodes)}
    for u, v in edges:
        i, j = node_map[u], node_map[v]
        adj_matrix[i][j] = 1
        adj_matrix[j][i] = 1
    return adj_matrix, nodes

## Graph Traversal: DFS

Depth-First Search (DFS) is a graph traversal algorithm where we start at some node and then visit all of its neighbors. We then visit all of the neighbors of the neighbors, and so on. We keep track of which nodes we've visited by using a set. We can use recursion to implement DFS, or we can use a stack. Let's see how we can implement DFS using recursion.

```shell
FUNC dfs_recursive(graph, start_node, visited):
    IF start_node is not in visited:
        ADD start_node to visited
        FOR each neighbor of start_node:
            CALL dfs_recursive(graph, neighbor, visited)
```

for using a stack,

```shell
FUNC dfs_iterative(graph, start_node):
    CREATE a stack
    PUSH start_node onto the stack
    WHILE the stack is not empty:
        POP a node off the stack
        IF the node has not been visited:
            MARK the node as visited
            FOR each neighbor of the node:
                PUSH the neighbor onto the stack
```


In [ ]:
# DFS Recursive
def dfs_recursive(adj_map, node, visited):
    if node in visited:
        return
    visited.add(node)
    for v in adj_map[node]:
        dfs_recursive(adj_map, v, visited)

In [ ]:
# DFS Iterative
def dfs_iterative(adj_map, start):
    visited = set()
    stack = [start]
    while stack:
        u = stack.pop()
        if u in visited:
            continue
        visited.add(u)
        for v in adj_map[u]:
            stack.append(v)

### DFS: Mental Model

I like to think of DFS as carving out large slices thru a graph in an almost greedy way. If we're observing someone in a maze, they're basically keeping to the right every time they come to a fork. Eventually they'll find their way out no matter what. Compared to BFS, this algorithm is much faster. However, the downsides is that this algorithm may not be used for Optimality problems like finding Min/Max values in a graph.

## Graph Traversal: BFS

Breadth-First Search (BFS) is a graph traversal algorithm where we start at some node and then visit all of its neighbors. We then visit all of the neighbors of the neighbors, and so on. We keep track of which nodes we've visited by using a set. We can use a queue to implement BFS. This small but significant distinction means we prioritize observing all the current unknown nodes, before we add more unknown nodes to the Queue. Let's see how we can implement BFS using a queue.

```shell
FUNC bfs_recursive(graph, queue, visited):
    IF the queue is empty:
        RETURN
    DEQUEUE a node off the queue
    IF the node has not been visited:
        MARK the node as visited
        FOR each neighbor of the node:
            ENQUEUE the neighbor onto the queue
    CALL bfs_recursive(graph, queue, visited)
```

It should be noted that we need a global queue for BFS. This is because there's a non-stack order to the subsequent function calls. In DFS, the execution-call-stack acts as an implied data structure that the DFS algorithm doesn't need to explicitly handle/manage. Such is not the case for BFS, therefore, there's really no benefit to using recursion for BFS, in fact it's slightly **less performant** than an iterative solution, since there's additional response overhead for call-stack operations.

```shell
FUNC bfs_iterative(graph, start_node):
    CREATE a queue
    ENQUEUE start_node onto the queue
    WHILE the queue is not empty:
        DEQUEUE a node off the queue
        IF the node has not been visited:
            MARK the node as visited
            FOR each neighbor of the node:
                ENQUEUE the neighbor onto the queue
```


In [ ]:
# BFS Iterative
from collections import deque


def bfs_iterative(graph, start_node):
    visited = set()
    pQ = deque([start_node])
    while pQ:
        u = pQ.pop()
        if u in visited:
            continue
        visited.add(u)
        for v in graph[u]:
            pQ.appendLeft(v)

### BFS: Mental Model

Slow methodical spread. Like a fungal virus that spreads outwards in a circle. This algorithm is great for finding the shortest path between two nodes. It's also great for finding the optimal solution to a problem. However, it's also great for finding the maximum or minimum value in a graph, that's because we need to know every single node in the graph before we can make a decision. This is the opposite of DFS, which is great for finding the first solution to a problem, but not necessarily the optimal solution.

## Graph Attributes: Connected Components

Knowing how connected a graph is
